In [27]:
import random
import struct
import os
import time
start = time.time()
def generate_binary_file(file_path, num_values):
    with open(file_path, 'wb') as file:
        for _ in range(num_values):
            # Генерируем случайное беззнаковое 32-разрядное целое число
            random_int = random.randint(0, 2**32 - 1)
            # Переводим число в байтовую строку в формате big endian
            byte_data = struct.pack('>I', random_int)
            # Записываем байтовую строку в файл
            file.write(byte_data)

# Задаем путь к файлу и количество чисел для генерации
file_path = 'random_numbers.bin'
num_values = 599000000  # Генерируем 599 миллионов чисел, чтоб объем был не менее 2 Гб

generate_binary_file(file_path, num_values)

finish = time.time()

file_info = os.stat(file_path)
file_size_bytes = file_info.st_size


file_size_kb = file_size_bytes / 1024 # Переводим в Кб
file_size_mb = file_size_kb / 1024 # перевоводим в Мб
file_size_gb = file_size_mb / 1024 # Переводим в Гб


res = finish - start
res_sec = float(res / 60)# Записываем время в минутах

print(f"Бинарный файл '{file_path}' создан.")
print(f"Размер файла '{file_path}': {file_size_gb:.2f} Гб")
print(f"Время работы : {res_sec:.1f} минут")



Бинарный файл 'random_numbers.bin' создан.
Размер файла 'random_numbers.bin': 2.23 Гб
Время работы : 18.2 минут


In [28]:
import struct
import time

start = time.time() 
def read_binary_file(file_path):
    total_sum = 0
    min_number = float('inf')
    max_number = float('-inf')

    with open(file_path, 'rb') as file:
        while True:
            # Читаем 4 байта из файла в формате big endian 
            byte_data = file.read(4)
            if not byte_data:
                break 

            # Распаковываем байтовую строку в число в формате big endian
            number = struct.unpack('>I', byte_data)[0]

            # Записываем(обновляем) сумму, минимальное и максимальное число
            total_sum += number
            min_number = min(min_number, number)
            max_number = max(max_number, number)

    return total_sum, min_number, max_number

# Путь к файлу
file_path = 'random_numbers.bin'

# Читаем файл и находим сумму, минимальное и максимальное число
total_sum, min_number, max_number = read_binary_file(file_path)


finish = time.time()
res = finish - start
res_sec = float(res / 60)# Записываем время в минутах


print("Сумма чисел:", total_sum)
print("Минимальное число:", min_number)
print("Максимальное число:", max_number)
print(f"Время работы : {res_sec:.1f} минут")


Сумма чисел: 1286373946426416511
Минимальное число: 2
Максимальное число: 4294967292
Время работы : 7.2 минут


In [1]:
import mmap
import os
import struct
import concurrent.futures
import time

start = time.time()
def read_binary_file_multithreaded(file_path):
    total_sum = 0
    min_number = float('inf')
    max_number = float('-inf')

    # Определяем размер файла
    file_size = os.path.getsize(file_path)

    # Открываем файл для чтения и memory-mapped файл
    with open(file_path, 'r+b') as file:
        with mmap.mmap(file.fileno(), length=file_size, access=mmap.ACCESS_READ) as mmapped_file:
            # Определяем количество потоков
            num_threads = os.cpu_count()

            # Создаем функцию для каждого потока
            def process_chunk(start, end):
                nonlocal total_sum, min_number, max_number

                for i in range(start, end, 4):
                    # Читаем 4 байта из memory-mapped файла
                    byte_data = mmapped_file[i:i+4]

                    # Если дошли до конца файла, то прерываем выполнение
                    if not byte_data:
                        break

                    # Распаковываем байтовую строку в число  в формате big endian 
                    number = struct.unpack('>I', byte_data)[0]

                    # Записываем (обновляем) сумму, минимальное и максимальное число
                    total_sum += number
                    min_number = min(min_number, number)
                    max_number = max(max_number, number)

            # Вычисляем размер каждого "куска" данных для обработки каждым потоком
            chunk_size = file_size // num_threads
            start_positions = [i * chunk_size for i in range(num_threads)]
            end_positions = start_positions[1:] + [file_size]

            # Создаем пул потоков и выполняем чтение в каждом потоке
            with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
                executor.map(process_chunk, start_positions, end_positions)

    return total_sum, min_number, max_number

# Путь к бинарному файлу
file_path = 'random_numbers.bin'

# Читаем файл и находим сумму, минимальное и максимальное число
total_sum, min_number, max_number = read_binary_file_multithreaded(file_path)

finish = time.time()
res = finish - start
res_sec = float(res / 60)# Записываем время в минутах

print("Сумма чисел (многопоточная версия):", total_sum)
print("Минимальное число (многопоточная версия):", min_number)
print("Максимальное число (многопоточная версия):", max_number)
print(f"Время работы : {res_sec:.1f} минут")

Сумма чисел (многопоточная версия): 1286373946426416511
Минимальное число (многопоточная версия): 2
Максимальное число (многопоточная версия): 4294967255
Время работы : 8.4 минут
